In [ ]:
import tensorflow as tf
import pickle
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Layer
from tensorflow.keras.utils import to_categorical
import numpy as np

In [ ]:
def load_cifar10_batch(file):
    with open(file, 'rb') as f:
        dict = pickle.load(f, encoding='bytes')
        X = dict[b'data']
        Y = dict[b'labels']
        X = X.reshape(10000, 3, 32, 32).transpose(0, 2, 3, 1).astype("float")
        Y = np.array(Y)
        return X, Y

# Load all training batches
X_train = []
Y_train = []
for i in range(1, 6):
    X_batch, Y_batch = load_cifar10_batch(f'\cifar-10-batches-py\data_batch_{i}')
    X_train.append(X_batch)
    Y_train.append(Y_batch)

X_train = np.concatenate(X_train, axis=0)
Y_train = np.concatenate(Y_train, axis=0)

# Load test batch
X_test, Y_test = load_cifar10_batch('cifar-10-batches-py\test_batch')

# Normalize pixel values to be between 0 and 1
X_train, X_test = X_train / 255.0, X_test / 255.0

# Convert class vectors to binary class matrices (one-hot encoding)
Y_train = to_categorical(Y_train, 10)
Y_test = to_categorical(Y_test, 10)

We have to implement a Local Response Normalization (LRN) Layer, which is used in AlexNet but is not predefined in keras.